# Validation ROC AUC:

In [ ]:
!pip install numpy==1.26.4
!pip install pandas==2.2.2
!pip install scikit-learn==1.3.2
!pip install catboost==1.2.7

In [ ]:
!pip install tqdm

In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

# 컬럼 타입 정의
CATEGORY_COLUMNS = ["시술 시기 코드", "시술 유형", "특정 시술 유형", "배란 유도 유형", 
                   "배아 생성 주요 이유", "난자 출처", "정자 출처"]
NUMERIC_COLUMNS = ["시술 당시 나이", "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수",
                  "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수",
                  "난자 기증자 나이", "정자 기증자 나이"]
COUNT_COLUMNS = ["총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수"]

def convert_count_str(val):
    """횟수 문자열을 숫자로 변환하는 함수"""
    if pd.isna(val):
        return 0.0
    val = str(val).strip()
    if "회 이상" in val:
        return 6.0
    m = re.search(r'(\d+)회?', val)
    if m:
        return float(m.group(1))
    return 0.0

def preprocess_dataframe(df):
    """데이터 프레임을 전처리하는 함수"""
    df_processed = df.copy()
    
    # 카테고리형 변수 처리
    for col in CATEGORY_COLUMNS:
        df_processed[col] = df_processed[col].fillna('Unknown').astype('category')

    # 숫자형 변수 처리
    for col in NUMERIC_COLUMNS:
        df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce').fillna(0).astype(float)

    # 횟수 관련 컬럼 변환
    for col in COUNT_COLUMNS:
        df_processed[col] = df_processed[col].astype(str).apply(convert_count_str).astype(float)

    return df_processed

def apply_feature_weights(X, weight_data):
    """모든 특성에 대해 시술 유형별 가중치를 적용하는 함수"""
    X_weighted = X.copy()
    
    # 가중치 데이터를 딕셔너리 형태로 변환
    weight_dict = {}
    for _, row in weight_data.iterrows():
        feature = row['데이터 항목']
        weights = row.drop('데이터 항목').to_dict()
        weight_dict[feature] = weights
    
    # 각 행에 대해 가중치 적용
    for idx, row in X_weighted.iterrows():
        procedure_type = row['특정 시술 유형']
        
        # 각 특성에 대해 가중치 적용
        for feature in weight_dict:
            if feature in X_weighted.columns:
                try:
                    current_value = float(X_weighted.loc[idx, feature])
                    
                    # 시술 유형에 따른 가중치 선택
                    if procedure_type in weight_dict[feature]:
                        weight = float(weight_dict[feature][procedure_type])
                    else:
                        weight = float(weight_dict[feature]['Unknown'])
                    
                    # DI 관련 특성은 IUI일 때만 의미있는 값을 가짐
                    if 'DI' in feature and procedure_type != 'IUI':
                        weight = 0.0
                        
                    # 가중치 적용
                    X_weighted.loc[idx, feature] = current_value * weight
                    
                except (ValueError, TypeError):
                    continue
    
    return X_weighted

# 1. 데이터 로드
print("=== 데이터 로드 중... ===")
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
weight_data = pd.read_csv('new_weighted_hong.csv', encoding='euc-kr')

# 2. 데이터 탐색
print("\n=== 데이터 기본 정보 ===")
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

# 시술 유형별 성공률
print("\n=== 시술 유형별 성공률 ===")
success_by_type = train.groupby('시술 유형')['임신 성공 여부'].agg(['count', 'mean'])
success_by_type.columns = ['건수', '성공률']
print(success_by_type)

# 3. 데이터 전처리
print("\n=== 데이터 전처리 시작 ===")
train_processed = preprocess_dataframe(train)
test_processed = preprocess_dataframe(test)

# 4. 가중치 적용
X = train_processed.drop(['ID', '임신 성공 여부'], axis=1)
y = train_processed['임신 성공 여부']
X_test = test_processed.drop('ID', axis=1)

X_weighted = apply_feature_weights(X, weight_data)
X_test_weighted = apply_feature_weights(X_test, weight_data)

# 5. 데이터 불균형 처리
print("\n=== 데이터 불균형 처리 ===")
undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = undersample.fit_resample(X_weighted, y)

# 6. 학습/검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# 7. 모델 학습
print("\n=== 모델 학습 시작 ===")
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    cat_features=[X_train.columns.get_loc(col) for col in CATEGORY_COLUMNS],
    random_seed=42,
    task_type='GPU',
    verbose=100
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50,
    verbose=True
)

# 8. 모델 평가
print("\n=== 모델 평가 ===")
val_predictions = model.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_predictions)
print(f"Validation AUC: {val_auc:.4f}")

# 9. 특성 중요도
print("\n=== 주요 특성 중요도 ===")
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print(feature_importance.head(10))

# 10. 예측 및 제출 파일 생성
test_predictions = model.predict_proba(X_test_weighted)[:, 1]
submission = pd.DataFrame({
    'ID': test['ID'],
    'probability': test_predictions
})
submission.to_csv('submission.csv', index=False)
print("\n=== 제출 파일 생성 완료 ===")

=== 데이터 로드 중... ===

=== 데이터 기본 정보 ===
Train shape: (256351, 69)
Test shape: (90067, 68)

=== 시술 유형별 성공률 ===
           건수       성공률
시술 유형                  
DI       6291  0.128914
IVF    250060  0.261605

=== 데이터 전처리 시작 ===

=== 데이터 불균형 처리 ===

=== 모델 학습 시작 ===


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7070430	best: 0.7070430 (0)	total: 115ms	remaining: 57.5s
1:	total: 148ms	remaining: 36.8s
2:	total: 178ms	remaining: 29.5s
3:	total: 210ms	remaining: 26s
4:	total: 241ms	remaining: 23.8s
5:	test: 0.7138028	best: 0.7138028 (5)	total: 271ms	remaining: 22.3s
6:	total: 314ms	remaining: 22.1s
7:	total: 347ms	remaining: 21.4s
8:	total: 381ms	remaining: 20.8s
9:	total: 413ms	remaining: 20.2s
10:	test: 0.7168368	best: 0.7168368 (10)	total: 445ms	remaining: 19.8s
11:	total: 480ms	remaining: 19.5s
12:	total: 513ms	remaining: 19.2s
13:	total: 545ms	remaining: 18.9s
14:	total: 575ms	remaining: 18.6s
15:	test: 0.7188379	best: 0.7188379 (15)	total: 605ms	remaining: 18.3s
16:	total: 634ms	remaining: 18s
17:	total: 665ms	remaining: 17.8s
18:	total: 696ms	remaining: 17.6s
19:	total: 731ms	remaining: 17.5s
20:	test: 0.7200610	best: 0.7200610 (20)	total: 771ms	remaining: 17.6s
21:	total: 808ms	remaining: 17.6s
22:	total: 838ms	remaining: 17.4s
23:	total: 869ms	remaining: 17.2s
24:	total: 901m

In [7]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.combine import SMOTETomek
import warnings
warnings.filterwarnings('ignore')

# 컬럼 타입 정의
CATEGORY_COLUMNS = ["시술 시기 코드", "시술 유형", "특정 시술 유형", "배란 유도 유형", 
                   "배아 생성 주요 이유", "난자 출처", "정자 출처"]
NUMERIC_COLUMNS = ["시술 당시 나이", "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수",
                  "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수",
                  "난자 기증자 나이", "정자 기증자 나이"]
COUNT_COLUMNS = ["총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수"]

def convert_count_str(val):
    """횟수 문자열을 숫자로 변환하는 함수"""
    if pd.isna(val):
        return 0.0
    val = str(val).strip()
    if "회 이상" in val:
        return 6.0
    m = re.search(r'(\d+)회?', val)
    if m:
        return float(m.group(1))
    return 0.0

def create_additional_features(df):
    """추가 피처 생성"""
    df_new = df.copy()
    
    # 연령대 구분 (직접 구간 정의)
    age_bins = [-np.inf, 25, 30, 35, 40, np.inf]
    age_labels = ['20대', '30대초반', '30대후반', '40대초반', '40대이상']
    df_new['연령_구간'] = pd.cut(df_new['시술 당시 나이'], 
                             bins=age_bins, 
                             labels=age_labels)
    
    # 시술 경험 지표
    df_new['총_시술_경험'] = df_new['IVF 시술 횟수'] + df_new['DI 시술 횟수']
    df_new['시술_성공률'] = df_new['총 임신 횟수'] / (df_new['총_시술_경험'] + 1)
    
    # 임신/출산 이력 지표
    df_new['이전_임신_경험'] = df_new['총 임신 횟수'] > 0
    df_new['이전_출산_경험'] = df_new['총 출산 횟수'] > 0
    
    # 시술 간 상호작용
    df_new['IVF_DI_비율'] = df_new['IVF 시술 횟수'] / (df_new['DI 시술 횟수'] + 1)
    
    # 나이 관련 상호작용
    df_new['난자_정자_나이차이'] = abs(df_new['난자 기증자 나이'] - df_new['정자 기증자 나이'])
    df_new['시술자_난자기증자_나이차이'] = abs(df_new['시술 당시 나이'] - df_new['난자 기증자 나이'])
    
    return df_new

def preprocess_dataframe(df):
    """데이터 프레임을 전처리하는 함수"""
    df_processed = df.copy()
    
    # 카테고리형 변수 처리
    for col in CATEGORY_COLUMNS:
        df_processed[col] = df_processed[col].fillna('Unknown').astype('category')

    # 숫자형 변수 처리
    for col in NUMERIC_COLUMNS:
        df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce').fillna(0).astype(float)

    # 횟수 관련 컬럼 변환
    for col in COUNT_COLUMNS:
        df_processed[col] = df_processed[col].astype(str).apply(convert_count_str).astype(float)

    # 추가 피처 생성
    df_processed = create_additional_features(df_processed)
    
    return df_processed

def apply_feature_weights(X, weight_data):
    """가중치 적용 함수"""
    X_weighted = X.copy()
    
    # 가중치 데이터를 딕셔너리 형태로 변환
    weight_dict = {}
    for _, row in weight_data.iterrows():
        feature = row['데이터 항목']
        weights = row.drop('데이터 항목').to_dict()
        weight_dict[feature] = weights
    
    # 각 행에 대해 가중치 적용
    for idx, row in X_weighted.iterrows():
        procedure_type = row['특정 시술 유형']
        
        # 각 특성에 대해 가중치 적용
        for feature in weight_dict:
            if feature in X_weighted.columns:
                try:
                    current_value = float(X_weighted.loc[idx, feature])
                    
                    if procedure_type in weight_dict[feature]:
                        weight = float(weight_dict[feature][procedure_type])
                    else:
                        weight = float(weight_dict[feature]['Unknown'])
                    
                    if 'DI' in feature and procedure_type != 'IUI':
                        weight = 0.0
                        
                    X_weighted.loc[idx, feature] = current_value * weight
                    
                except (ValueError, TypeError):
                    continue
    
    return X_weighted

# 데이터 로드
print("데이터 로드 중...")
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])
weight_data = pd.read_csv('new_weighted_hong.csv', encoding='euc-kr')

# 전처리 적용
print("데이터 전처리 중...")
train_processed = preprocess_dataframe(train)
test_processed = preprocess_dataframe(test)

# Feature 가중치 적용
X = train_processed.drop('임신 성공 여부', axis=1)
y = train_processed['임신 성공 여부']
X_weighted = apply_feature_weights(X, weight_data)
X_test_weighted = apply_feature_weights(test_processed, weight_data)

# 데이터 불균형 처리
print("데이터 불균형 처리 중...")
resampler = SMOTETomek(random_state=42)
X_resampled, y_resampled = resampler.fit_resample(X_weighted, y)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# 범주형 변수의 인덱스 가져오기
cat_features_idx = [list(X_train.columns).index(col) for col in CATEGORY_COLUMNS]

# 스태킹 모델 정의
print("모델 구성 중...")
stack_clf = StackingClassifier(
    estimators=[
        ('xgb1', XGBClassifier(
            tree_method='hist',
            device='cuda',
            enable_categorical=True,
            random_state=42
        )),
        ('xgb2', XGBClassifier(
            tree_method='hist',
            device='cuda',
            enable_categorical=True,
            random_state=43
        )),
        ('lgbm1', LGBMClassifier(
            n_jobs=1,
            random_state=42,
            verbose=-1,
            categorical_feature=CATEGORY_COLUMNS
        )),
        ('lgbm2', LGBMClassifier(
            n_jobs=1,
            random_state=43,
            verbose=-1,
            categorical_feature=CATEGORY_COLUMNS
        )),
        ('cat1', CatBoostClassifier(
            cat_features=cat_features_idx,
            task_type='GPU',
            verbose=0,
            random_seed=42
        )),
        ('cat2', CatBoostClassifier(
            cat_features=cat_features_idx,
            task_type='GPU',
            verbose=0,
            random_seed=43
        ))
    ],
    final_estimator=Pipeline([
        ('scaler', RobustScaler()),
        ('lr', LogisticRegression(max_iter=2000, class_weight='balanced', solver='liblinear'))
    ]),
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=1
)

# 하이퍼파라미터 튜닝
param_dist = {
    'xgb1__n_estimators': [300, 400, 500, 600],
    'xgb1__max_depth': [4, 5, 6, 7],
    'xgb1__learning_rate': [0.01, 0.025, 0.05, 0.1],
    'xgb1__min_child_weight': [1, 3, 5],
    'xgb1__subsample': [0.8, 0.9, 1.0],
    'xgb1__colsample_bytree': [0.8, 0.9, 1.0],
    
    'xgb2__n_estimators': [300, 400, 500, 600],
    'xgb2__max_depth': [4, 5, 6, 7],
    'xgb2__learning_rate': [0.01, 0.025, 0.05, 0.1],
    'xgb2__min_child_weight': [1, 3, 5],
    'xgb2__subsample': [0.8, 0.9, 1.0],
    'xgb2__colsample_bytree': [0.8, 0.9, 1.0],
    
    'lgbm1__n_estimators': [300, 400, 500, 600],
    'lgbm1__num_leaves': [31, 63, 127],
    'lgbm1__min_child_samples': [20, 30, 50],
    'lgbm1__learning_rate': [0.01, 0.025, 0.05, 0.1],
    'lgbm1__subsample': [0.8, 0.9, 1.0],
    'lgbm1__colsample_bytree': [0.8, 0.9, 1.0],
    
    'lgbm2__n_estimators': [300, 400, 500, 600],
    'lgbm2__num_leaves': [31, 63, 127],
    'lgbm2__min_child_samples': [20, 30, 50],
    'lgbm2__learning_rate': [0.01, 0.025, 0.05, 0.1],
    'lgbm2__subsample': [0.8, 0.9, 1.0],
    'lgbm2__colsample_bytree': [0.8, 0.9, 1.0],
    
    'cat1__iterations': [300, 500, 700],
    'cat1__depth': [4, 5, 6, 7],
    'cat1__learning_rate': [0.01, 0.025, 0.05, 0.1],
    'cat1__l2_leaf_reg': [3, 5, 7],
    'cat1__subsample': [0.8, 0.9, 1.0],
    
    'cat2__iterations': [300, 500, 700],
    'cat2__depth': [4, 5, 6, 7],
    'cat2__learning_rate': [0.01, 0.025, 0.05, 0.1],
    'cat2__l2_leaf_reg': [3, 5, 7],
    'cat2__subsample': [0.8, 0.9, 1.0],
    
    'final_estimator__lr__C': [0.1, 0.5, 1.0, 2.0, 5.0]
}

random_search = RandomizedSearchCV(
    stack_clf,
    param_distributions=param_dist,
    n_iter=100,
    scoring='roc_auc',
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=1,
    random_state=42,
    verbose=2
)

# 모델 학습
print("\n=== 모델 학습 시작 ===")
random_search.fit(X_train, y_train)

# 검증 데이터 평가
print("\n=== 모델 평가 ===")
y_val_pred = random_search.best_estimator_.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
print(f"Validation ROC AUC: {roc_auc:.5f}")

# 특성 중요도 분석
print("\n=== 특성 중요도 (XGBoost 기준) ===")
xgb_model = random_search.best_estimator_.named_estimators_['xgb1']
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)
print(importance_df.head(10))

# 최적 모델로 테스트 데이터 예측
print("\n=== 테스트 데이터 예측 중 ===")
pred_proba = random_search.best_estimator_.predict_proba(X_test_weighted)[:, 1]
submission = pd.DataFrame({
    'ID': [f"TEST_{i:05d}" for i in range(len(test))],
    'probability': pred_proba
})

# 제출 파일 저장
submission.to_csv('improved_stack_model_submit.csv', index=False)
print("\n=== 제출 파일 생성 완료 ===")

# 모델 성능 및 파라미터 상세 출력
print("\n=== 모델 상세 정보 ===")
print(f"Best Validation Score: {random_search.best_score_:.5f}")
print(f"Best Parameters:")
for param, value in random_search.best_params_.items():
    print(f"{param}: {value}")

# 모든 교차 검증 점수 확인
print("\n=== 교차 검증 점수 ===")
cv_results = pd.DataFrame(random_search.cv_results_)
best_idx = random_search.best_index_
print("Best Model CV Scores:")
for i, score in enumerate(cv_results.loc[best_idx, [f'split{i}_test_score' for i in range(5)]], 1):
    print(f"Fold {i}: {score:.5f}")

print(f"\nCV Mean: {cv_results.loc[best_idx, 'mean_test_score']:.5f}")
print(f"CV Std: {cv_results.loc[best_idx, 'std_test_score']:.5f}")

# 학습 시간 정보
print("\n=== 학습 시간 정보 ===")
total_time = cv_results.loc[best_idx, 'mean_fit_time'] * 5  # 5-fold CV
print(f"Average Fit Time per Fold: {cv_results.loc[best_idx, 'mean_fit_time']:.2f} seconds")
print(f"Total Fit Time for Best Model: {total_time:.2f} seconds")

# 추가 분석: 예측 분포 확인
print("\n=== 예측 분포 분석 ===")
print("Test Predictions Statistics:")
print(pd.Series(pred_proba).describe())

# 모델 저장 (선택적)
import joblib
joblib.dump(random_search.best_estimator_, 'best_stacking_model.joblib')
print("\n=== 모델 저장 완료 ===")

print("\n=== 분석 완료 ===")

데이터 로드 중...
데이터 전처리 중...
데이터 불균형 처리 중...


ValueError: Cannot cast object dtype to float64